In [5]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd
import time
import math
import re

In [6]:
client_credentials = [
('ef58993d4fbe446ca6a6edd303a56c80', '3fe8edaf5c8147ff9e81ea858b410102'),
('e857908311cd436abed7fe2b1dc99bd5', '56411c35c3874b9ebbaf3f3fe80a811c'),
('470adf495fd548298483dc4d1a38b1e2', 'c16d7144415946028ae38b6d635f6bfd'),
('adafeb81352e4bddb7d5f3be5956ada8', 'a09dbec853264e94a0a5f61eb6df67ed'),
('4c42335d9dc74a36a48cee122449c9d6', '7b0c7b87143d4051b0e3891e70e83da9'),
('ed1bfedbe22244fdbc9f9a592bb8b153', '50585f3a59364e48980e5f576ec9d719'),
('dc3085808e9e490a8426ec00952e12c9', '4b9c26457b034c79b8363ed19ebd9e30'),
('af8d0d58749e48c19ac32f47a8089e31', 'c021220c33f94bd6bceec431617564c8'),
('200886fa3780485faa62b4e89034f7b8', '0deb41f8c0684e619ebceff7e7160d44'),
('46a05bf1cd064188a47c2ec88aca1aeb', 'ca9ce67a39a44b81854d64c9a6acb2bd')

]



credential_index = 0

def get_token(client_id, client_secret):
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_albums_for_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    params = {"include_groups": "album"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        albums = json.loads(response.text)["items"]
        album_dict = {}
        for album in albums:
            album_id = album["id"]
            album_name = album["name"]
            album_tracks = album["total_tracks"]
            album_type = album["album_type"]
            album_release_date = album["release_date"]
            album_artists = [artist["name"] for artist in album["artists"]]
            album_images = album["images"]

            if album_name in album_dict:
                existing_tracks = album_dict[album_name]["total_tracks"]
                if album_tracks > existing_tracks:
                    album_dict[album_name] = {
                        "album_id": album_id,
                        "album_name": album_name,
                        "total_tracks": album_tracks,
                        "album_type": album_type,
                        "release_date": album_release_date,
                        "artists": album_artists,
                        "images": album_images
                    }
            else:
                album_dict[album_name] = {
                    "album_id": album_id,
                    "album_name": album_name,
                    "total_tracks": album_tracks,
                    "album_type": album_type,
                    "release_date": album_release_date,
                    "artists": album_artists,
                    "images": album_images
                }
        return album_dict
    else:
        print("Error: Failed to get albums for artist")
        print(response.content)
        return None
    
def get_tracks_for_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    params = {"market": "US"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks = json.loads(response.text)["items"]
        track_list = []
        for track in tracks:
            track_name = track["name"]
            track_id = track["id"]
            track_number = track["track_number"]
            track_explicit = track["explicit"]
            track_artists = [artist["name"] for artist in track["artists"]]
            track_dict = {
                "track_name": track_name,
                "track_id": track_id,
                "track_number": track_number,
                "track_explicit": track_explicit,
                "track_artists": track_artists,
            }
            track_list.append(track_dict)
        return track_list
    else:
        print("Error: Failed to get tracks for album")
        print(response.content)
        return None
    
def get_audio_features(token, track_ids):
    audio_features = []

    for i in range(0, len(track_ids), 50):
        chunk = track_ids[i:i + 50]
        ids = ','.join(chunk)
        url = f"https://api.spotify.com/v1/audio-features?ids={ids}"
        headers = {"Authorization": f"Bearer {token}"}
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            print(f"Error getting audio features for tracks: {r.status_code}")
            return None

        try:
            features = r.json()["audio_features"]
        except KeyError:
            features = []

        audio_features.extend(features)

    audio_features_filtered = []
    for feature in audio_features:
        if feature is not None:
            audio_features_filtered.append(feature)

    return pd.DataFrame(audio_features_filtered)

def load_checkpoint():
    if os.path.exists('checkpoint_f1.json'):
        with open('checkpoint_f1.json', 'r') as f:
            return json.load(f)
    else:
        return {"artist_id": "", "progress": 0}

def save_checkpoint(artist_id, progress):
    with open('checkpoint_f1.json', 'w') as f:
        json.dump({"artist_id": artist_id, "progress": progress}, f)

def load_failed_requests():
    if os.path.exists('failed_requests_f1.parquet'):
        return pd.read_parquet('failed_requests_f1.parquet')
    else:
        return pd.DataFrame(columns=['Spotify ID'])

def save_failed_requests(df):
    df.to_parquet('failed_requests_f1.parquet')

def refresh_token(token_start_time):
    global credential_index
    current_time = time.time()
    elapsed_time = current_time - token_start_time
    if elapsed_time >= 1800:
        credential_index = (credential_index + 1) % len(client_credentials)
        client_id, client_secret = client_credentials[credential_index]
        return get_token(client_id, client_secret), current_time
    return None, token_start_time

def sanitize_filename(filename):
    return re.sub(r'[\\/:"*?<>|]+', '_', filename)

In [7]:
with open('lost_artist_ids_sub_dict_1.json', 'r') as f:
    artist_ids = json.load(f)
len(artist_ids)

3202

In [8]:
token = get_token(client_credentials[0][0], client_credentials[0][1])
token_start_time = time.time()

checkpoint = load_checkpoint()
failed_requests = load_failed_requests()

start_from = checkpoint["artist_id"] if checkpoint["artist_id"] else list(artist_ids.keys())[0]
start_index = list(artist_ids.keys()).index(start_from)


iteration_counter = 0
sleep_interval = 50

for artist_id, artist_name in list(artist_ids.items())[start_index:]:
    sanitized_artist_name = sanitize_filename(artist_name)
    parquet_file_path = f'results/{sanitized_artist_name}_tracks.parquet'
    if os.path.exists(parquet_file_path):
        print(f"Parquet file already exists for {artist_name}, skipping...")
        continue
    try:
        refreshed_token, new_token_start_time = refresh_token(token_start_time)
        if refreshed_token:
            token = refreshed_token
            token_start_time = new_token_start_time

        albums = get_albums_for_artist(token, artist_id)

        if albums is None:
            failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
            save_failed_requests(failed_requests)
            continue

        album_df = pd.DataFrame.from_dict(albums, orient="index")
        album_df.index.name = "album_name"

        all_tracks_df = pd.DataFrame()
        for album_id in tqdm(album_df['album_id'], desc=f"Processing {artist_name}"):
            time.sleep(1)
            tracks = get_tracks_for_album(token, album_id)
            if tracks is None:
                failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
                save_failed_requests(failed_requests)
                continue

            tracks = pd.DataFrame(tracks)
            tracks['album_id'] = album_id
            all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)

        af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
        af_df = af_df.rename(columns={'id': 'track_id'})
        all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
        album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
        sanitized_artist_name = sanitize_filename(artist_name)
        album_all_aftracks_df.to_parquet(f'results/{sanitized_artist_name}_tracks.parquet')

        save_checkpoint(artist_id, artist_name)
        iteration_counter += 1
        if iteration_counter % sleep_interval == 0:
            time.sleep(10)
            
    except Exception as e:
        print(f"Error processing {artist_name}: {e}")
        failed_requests = failed_requests.append({"Spotify ID": artist_id}, ignore_index=True)
        save_failed_requests(failed_requests)
        continue

print("Completed processing all artists.")

Parquet file already exists for The Radiators, skipping...
Parquet file already exists for Hungry Lucy, skipping...
Parquet file already exists for Judy Kuhn, skipping...
Parquet file already exists for Melanie Fiona, skipping...
Parquet file already exists for Michael Hurley, skipping...
Parquet file already exists for Louise Goffin, skipping...
Parquet file already exists for Candi Staton, skipping...
Parquet file already exists for California X, skipping...
Parquet file already exists for Archie Fisher, skipping...
Parquet file already exists for Roadkill Ghost Choir, skipping...
Parquet file already exists for Josh Roseman Unit, skipping...
Parquet file already exists for Roger Waters, skipping...
Parquet file already exists for Rogue Wave, skipping...
Parquet file already exists for Phil Collins, skipping...
Parquet file already exists for Nashville Pussy, skipping...
Parquet file already exists for Imani Coppola, skipping...
Parquet file already exists for The Beguiled, skipping.